## Injection benchmark

### Setup

In [1]:
import sys
from antidote import __version__, is_compiled
print(f"Antidote: {__version__()} {'(cython)' if is_compiled() else ''}")
print(f"Python {sys.version}")

Antidote: 0.7.4.dev4+g5ddfb12.d20200822 
Python 3.8.5 (default, Aug 21 2020, 11:31:30) 
[GCC 9.3.0]


In [2]:
from antidote import world, Service, inject

class Service1(Service):
    pass


class Service2(Service):
    def __init__(self, service1: Service1):
        self.service1 = service1


class Service3(Service):
    __antidote__ = Service.Conf(singleton=False)
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2


class Service4(Service):
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

In [3]:
from antidote._internal.wrapper import get_wrapper_dependencies
from antidote._inten
from collections import deque
import textwrap

def debug_get_injected_dependencies(func):
    try:
        return get_wrapper_dependencies(func)
    except TypeError:
        return []
    
class Node:
    
    
def tree_debug_info(dependency,
                    depth: int,
                    helper_txt: bool) -> str:
    from antidote._internal.state import get_container
    container = get_container()
    providers = container.providers
    nodes: List[Tuple[int, int, DependencyDebug, bool]] = []
    q = deque([(0, 0, dependency, True)])

    while q:
        mask, level, dep, last = q.pop()
        for p in providers:
            debug = p.maybe_debug(dep)
            if debug is not None:
                nodes.append((mask, level, debug, last))
                children_mask = mask | (1 << level if not last else 0)
                if level < depth:
                    for i, child in enumerate(debug.dependencies[::-1]):
                        q.append((children_mask, level + 1, child, i == 0))
                    for 

    info_tree = []
    for mask, level, debug, last in nodes:
        if level == 0:
            info_tree.append(("" if debug.singleton else "* ")
                             + textwrap.dedent(debug.info))
        else:
            first_line, *rest = textwrap.dedent(debug.info).split("\n", 1)
            txt = ("└─" if last else "├─") \
                  + ("─ " if debug.singleton else "* ") \
                  + first_line
            if rest:
                txt += "\n" + textwrap.indent(rest, "    " if last else "│   ")

            indent = "".join(
                ("│   " if ((1 << i) & mask == (1 << i)) else "    ")
                for i in range(1, level)
            )
            info_tree.append(textwrap.indent(txt, indent))

    if helper_txt:
        info_tree.append("\n* = not a singleton")
    return "\n".join(info_tree)

print(tree_debug_info(Service4, depth=10, helper_txt=True))

Service4 

* = not a singleton


### Results
The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the most common use case of dependency injection, service instantiation, the overhead should be negligible.

It should be noted that in most cases the worst scenario is used, as functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [2]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

2.45 µs ± 50.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
print(world.debug.info(Service3, recursive=True))

* Service3 
├── Service1 
└── Service2 
    └── Service1 

* = not a singleton


### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

Time necessary to only execute the function, without retrieving the services

In [5]:
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
# %timeit f(*args)

Overhead of the injection when all argument must be retrieved from the container.

In [9]:
f_injected = inject(f)
assert f(*args) == f_injected()
%timeit f_injected()

450 ns ± 9.55 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [8]:
assert f(*args) == f_injected(*args)
%timeit f_injected(*args)

183 ns ± 8.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Method call

In [9]:
class Dummy:
    @inject
    def f(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4

In [10]:
%timeit Dummy().f(*args)

426 ns ± 3.76 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
d = Dummy()
assert d.f(*args) == d.f()
%timeit d.f()

611 ns ± 2.86 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Non singleton

In [11]:
class ServiceX(Service):
    __antidote__ = Service.Conf(singleton=False)

In [12]:
%timeit world.get(ServiceX)

489 ns ± 4.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Factory

In [13]:
from antidote import factory

class ServiceF:
    pass

@factory(singleton=False)
def f() -> ServiceF:
    return ServiceF()

In [14]:
%timeit f()

333 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
fid = ServiceF @ f  # This is relatively slow 1-2us
%timeit world.get(fid)

604 ns ± 33.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Custom Provider

In [16]:
from antidote import world
from antidote.core import Provider, DependencyInstance
dep = object()
dep2 = object()

@world.provider
class SlowProvider(Provider):
    def provide(self, dependency, container):
        if dep2 is dependency:
            return DependencyInstance("sleepy")
            

@world.provider
class CustomProvider(Provider):
    def provide(self, dependency, container):
        if dependency is dep:
            return DependencyInstance("yeah")


In [17]:
%timeit world.get(dep)

640 ns ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

In [12]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

437 ns ± 12.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
class ObjInjected:
    @inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

954 ns ± 8.43 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Configuration


In [ ]:
from antidote import LazyConstants

class Conf(LazyConstants):
    A = 'A'
    B = 'B'

    def get(self, key):
        return key

In [ ]:
def g(a, b):
    return a, b

%timeit g('A', 'B')

In [ ]:
conf = Conf()
%timeit g(conf.get('A'), conf.get('B'))

In [ ]:
assert (conf.A, conf.B) == (conf.get('A'), conf.get('B'))
%timeit g(conf.A, conf.B)

In [ ]:
g_injected = inject(g, dependencies=(Conf.A, Conf.B))

assert g(conf.get('A'), conf.get('B')) == g_injected()
assert g(conf.A, conf.B) == g_injected()

%timeit g_injected()